# init

> Router assembly for Phase 1 selection routes

In [ ]:
#| default_exp routes.init

In [ ]:
#| export
from typing import List, Dict, Callable, Tuple

from fasthtml.common import APIRouter

from cjm_transcript_source_select.models import SelectionUrls
from cjm_transcript_source_select.routes.core import WorkflowStateStore
from cjm_transcript_source_select.services.source import SourceService
from cjm_transcript_source_select.routes.queue import init_queue_router
from cjm_transcript_source_select.routes.filtering import init_filtering_router
from cjm_transcript_source_select.routes.local_files import init_local_files_router
from cjm_transcript_source_select.routes.tabs import init_tabs_router

## Router Assembly

In [ ]:
#| export
def init_selection_routers(
    state_store: WorkflowStateStore,  # The workflow state store
    source_service: SourceService,  # The source service for queries
    workflow_id: str,  # The workflow identifier
    prefix: str,  # Base prefix for selection routes (e.g., "/workflow/selection")
) -> Tuple[List[APIRouter], SelectionUrls, Dict[str, Callable]]:  # (routers, urls, merged_routes)
    """Initialize and return all selection routers with URL bundle."""
    # Create a mutable URL bundle that gets populated after routes are defined
    # We need to create placeholder SelectionUrls first, then update it
    urls = SelectionUrls()

    # Initialize focused routers (they reference urls which will be populated below)
    queue_router, queue_routes = init_queue_router(
        state_store, workflow_id, source_service, f"{prefix}/queue", urls
    )
    filtering_router, filtering_routes = init_filtering_router(
        state_store, workflow_id, source_service, f"{prefix}/filtering", urls
    )
    local_files_router, local_files_routes = init_local_files_router(
        state_store, workflow_id, source_service, f"{prefix}/local_files", urls
    )
    tabs_router, tabs_routes = init_tabs_router(
        state_store, workflow_id, source_service, f"{prefix}/tabs", urls
    )

    # Now populate the URL bundle using .to() on route functions
    urls.add = queue_routes["add"].to()
    urls.remove = queue_routes["remove"].to()
    urls.reorder = queue_routes["reorder"].to()
    urls.clear = queue_routes["clear"].to()
    urls.select_all = queue_routes["select_all"].to()
    urls.preview = queue_routes["preview"].to()
    urls.toggle_focused = filtering_routes["toggle_focused"].to()
    urls.keyboard_reorder = filtering_routes["keyboard_reorder"].to()
    urls.filter = filtering_routes["filter"].to()
    urls.grouping_change = filtering_routes["grouping_change"].to()
    urls.browse_directory = local_files_routes["browse_directory"].to()
    urls.add_external = local_files_routes["add_external"].to()
    urls.remove_external = local_files_routes["remove_external"].to()
    urls.tab_switch = tabs_routes["tab_switch"].to()

    # Merge all routes for external access
    merged_routes = {
        **queue_routes,
        **filtering_routes,
        **local_files_routes,
        **tabs_routes,
    }

    routers = [queue_router, filtering_router, local_files_router, tabs_router]

    return routers, urls, merged_routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()